## Preparación

Constantes:

In [2]:
project = !gcloud config get-value project
PROJECT_ID = project[0] # Nombre del proyecto

REGION = 'us-central1' # Ubicación servidores a usar
EXPERIMENT = '05'
SERIES = '05'

# Parámetros BigQuery
BQ_PROJECT = PROJECT_ID # Proyecto de BigQuery
BQ_DATASET = 'fraud' # Nombre del dataset dentro de BigQuery
BQ_TABLE = 'fraud_prepped' # Nombre de tabla dentro del dataset

# Recuersos
DEPLOY_COMPUTE = 'n1-standard-4' # Tipo de hardware
DEPLOY_IMAGE='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest' # Imagen de Docker a usar

# Parámetros para entrenamiento de modelo
VAR_TARGET = 'Class' # Nombre (field) de las etiquetas en la tabla
VAR_OMIT = 'transaction_id' # Variables a omitir
EPOCHS = 4 # Épocas de entrenamiento
BATCH_SIZE = 100 # Batch size de entrenamiento

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S") # Timestamp para usar como identificador
BUCKET = PROJECT_ID # Bucket en Google Storage
URI = f"gs://{BUCKET}/{SERIES}/{EXPERIMENT}" # URI en Google Storage
DIR = f"temp/{EXPERIMENT}" # Path de carpeta temporal auxiliar

In [50]:
#!pip install tensorflow==2.10.0 tensorflow-io==0.27.0

Importaciones:

In [3]:
from google.cloud import bigquery

from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf

from google.cloud import aiplatform
from datetime import datetime
import os

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np
import pandas as pd
from sklearn import metrics as metrics

2023-09-28 23:25:27.949604: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 23:25:28.185143: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-28 23:25:29.206932: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-09-28 23:25:29.207114: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer

Declarar clientes de BigQuery y de Google Storage:

In [4]:
aiplatform.init(project = PROJECT_ID, location = REGION)
bq = bigquery.Client(project = PROJECT_ID)

Crear carpeta temporal auxiliar:

In [6]:
!rm -rf {DIR}
!mkdir -p {DIR}

## Datos de entrenamiento

Se usan los datos de transacciones bancarias importados en tabla en BigQuery.

### Esquema de la tabla

Recuperar información de las columnas de la tabla de datos.   
En BigQuery se puede usar `INFORMATION_SCHEMA` para obtener información sobre las columnas de una tabla, como el nombre y el tipo de dato que contiene.

In [13]:
query = f"SELECT * FROM {BQ_PROJECT}.{BQ_DATASET}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{BQ_TABLE}'"
schema = bq.query(query).to_dataframe()
schema

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position,collation_name,column_default,rounding_mode
0,statmike-mlops-349915,fraud,fraud_prepped,Time,1,YES,INT64,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
1,statmike-mlops-349915,fraud,fraud_prepped,V1,2,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
2,statmike-mlops-349915,fraud,fraud_prepped,V2,3,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
3,statmike-mlops-349915,fraud,fraud_prepped,V3,4,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
4,statmike-mlops-349915,fraud,fraud_prepped,V4,5,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
5,statmike-mlops-349915,fraud,fraud_prepped,V5,6,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
6,statmike-mlops-349915,fraud,fraud_prepped,V6,7,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
7,statmike-mlops-349915,fraud,fraud_prepped,V7,8,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
8,statmike-mlops-349915,fraud,fraud_prepped,V8,9,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
9,statmike-mlops-349915,fraud,fraud_prepped,V9,10,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None


### Número de clases
Obtener el número de clases distintas en el dataset programáticamente.    
Sabemos que son dos (fraudulenta o normal), pero código puede ser útil en otros dataset.

In [14]:
nclasses = bq.query(query = f'SELECT DISTINCT {VAR_TARGET} FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE} WHERE {VAR_TARGET} is not null').to_dataframe()
nclasses

,Class
0,0
1,1


### Preparación de columnas para TensorFlow I/O

Usar esquema de la tabla para preparar inputs para TensorFlow I/O:
- Crear lista con nombre de columnas a usar (omitir las innecesarias)
- Definir el tipo de datos de cada columna a partir de los *data type* entregados en el esquema.

In [17]:
OMIT = VAR_OMIT.split() + ['splits'] # Lista de columnas a omitir

selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist() # Lista de columnas a leer

# Lista con data type de las columnas a leer
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in schema[~schema.column_name.isin(OMIT)].data_type.tolist()]

## Leer data de tabla de BigQuery con TensorFlow I/O 

### Separar inputs en features y etiqueta

Crear función que separa datos entre features y etiqueta.   
Codifica en One-Hot las etiquetas.

In [18]:
def transTable(row_dict):
    target = row_dict.pop(VAR_TARGET)
    target = tf.one_hot(tf.cast(target,tf.int64), nclasses)
    target = tf.cast(target, tf.float32)
    return(row_dict, target)

### Función para leer batches con Tensorflow I/O

Función que lee batches de datos desde la tabla de BigQuery usando Tensorflow I/O.   
Se configura para leer data en paralelo para acelerar el entrenamiento.

In [19]:
def bq_reader(split):
    reader = BigQueryClient()

    training = reader.read_session(
        parent = f"projects/{PROJECT_ID}",
        project_id = BQ_PROJECT,
        table_id = BQ_TABLE,
        dataset_id = BQ_DATASET,
        selected_fields = selected_fields,
        output_types = output_types,
        row_restriction = f"splits='{split}'",
        requested_streams = 3
    )
    
    return training.parallel_read_rows(sloppy = True, num_parallel_calls = tf.data.experimental.AUTOTUNE)

Usar función para conjuntos de entrenamiento, validación y prueba.

In [22]:
train = bq_reader('TRAIN').prefetch(1).map(transTable).shuffle(BATCH_SIZE*10).batch(BATCH_SIZE)
validate = bq_reader('VALIDATE').prefetch(1).map(transTable).batch(BATCH_SIZE)
test = bq_reader('TEST').prefetch(1).map(transTable).batch(BATCH_SIZE)

### Revisar un batch de entrenamiento

In [24]:
for features, target in train.take(1):
    print('features:\n',list(features.keys()))
    print('\netiqueta:\n',target[0:10])

2023-09-28 23:26:28.048388: E tensorflow/core/framework/dataset.cc:580] UNIMPLEMENTED: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2023-09-28 23:26:28.048431: E tensorflow/core/framework/dataset.cc:584] UNIMPLEMENTED: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.


features:
 ['Amount', 'Time', 'V1', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V2', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9']

target:
 tf.Tensor(
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]], shape=(10, 2), dtype=float32)


## Definir modelo

Se crea modelo de regresión logística:

- Se definen los inputs del modelo.
- Se usa Batch Normalization para normalizar datos.
- Se construye modelo de regresión logistica con función de activación *softmax*.
- Se compila el modelo.

In [25]:
# Modelo de regresion logística

# Definición de input de modelo
feature_columns = {header: tf.feature_column.numeric_column(header) for header in selected_fields if header != VAR_TARGET}
feature_layer_inputs = {header: tf.keras.layers.Input(shape = (1,), name = header) for header in selected_fields if header != VAR_TARGET}

# Concatenar columnas individuales de features en capa única
feature_layer_outputs = tf.keras.layers.DenseFeatures(feature_columns.values(), name = 'feature_layer')(feature_layer_inputs)

# Paso de Batch normalization
normalized = tf.keras.layers.BatchNormalization(name = 'batch_normalization_layer')(feature_layer_outputs)

# Capa fully connected con activación softmax
logistic = tf.keras.layers.Dense(nclasses, activation = tf.nn.softmax, name = 'logistic')(normalized)

# Construcción de modelo
model = tf.keras.Model(
    inputs = feature_layer_inputs,
    outputs = logistic,
    name = EXPERIMENT
)

# Compilar modelo
model.compile(
    optimizer = tf.keras.optimizers.SGD(), #SGD or Adam
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy', tf.keras.metrics.AUC(curve = 'PR', name = 'auprc')]
)

In [28]:
model.summary()

Model: "05"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Amount (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 Time (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 V1 (InputLayer)                [(None, 1)]          0           []                               
                                                                                                  
 V10 (InputLayer)               [(None, 1)]          0           []                               
                                                                                                 

## Entrenamiento del modelo

- Entrenar modelo con método *fit*.
- Notar que se entrena en maquina local.

In [1]:
# Preparar logs de TensorBoard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = os.path.join(DIR, "logs", f'{TIMESTAMP}'), histogram_freq=1)
# Entrenar modelo
history = model.fit(train, epochs = EPOCHS, callbacks = [tensorboard_callback], validation_data = validate)

NameError: name 'tf' is not defined

## Evaluar el modelo

Evaluar poder de generalización del modelo usando los datos de prueba.

In [33]:
loss, accuracy, auprc = model.evaluate(test)

2023-09-28 23:29:21.556561: E tensorflow/core/framework/dataset.cc:580] UNIMPLEMENTED: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2023-09-28 23:29:21.556616: E tensorflow/core/framework/dataset.cc:584] UNIMPLEMENTED: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.


286/286 [==============================] - 3s 7ms/step - loss: 0.0056 - accuracy: 0.9993 - auprc: 0.9994


In [35]:
loss, accuracy, auprc = model.evaluate(validate)

2023-09-28 23:29:24.625741: E tensorflow/core/framework/dataset.cc:580] UNIMPLEMENTED: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2023-09-28 23:29:24.625795: E tensorflow/core/framework/dataset.cc:584] UNIMPLEMENTED: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.


283/283 [==============================] - 2s 7ms/step - loss: 0.0058 - accuracy: 0.9992 - auprc: 0.9995


In [37]:
loss, accuracy, auprc = model.evaluate(train)

2023-09-28 23:29:27.306970: E tensorflow/core/framework/dataset.cc:580] UNIMPLEMENTED: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2023-09-28 23:29:27.307036: E tensorflow/core/framework/dataset.cc:584] UNIMPLEMENTED: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.


2281/2281 [==============================] - 29s 13ms/step - loss: 0.0061 - accuracy: 0.9993 - auprc: 0.9994


In [39]:
predictions = model.predict(test)

actuals = np.empty(shape = [0, predictions.shape[1]])
for features, target in test.take(-1): # -1 indicates all batches
    actuals = np.append(actuals, target.numpy(), axis = 0)

predictions_proba = np.max(predictions, axis = 1)
predictions = np.argmax(predictions, axis = 1)
actuals = np.argmax(actuals, axis = 1)

2023-09-28 23:29:56.816335: E tensorflow/core/framework/dataset.cc:580] UNIMPLEMENTED: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2023-09-28 23:29:56.816393: E tensorflow/core/framework/dataset.cc:584] UNIMPLEMENTED: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.


286/286 [==============================] - 4s 7ms/step


2023-09-28 23:30:00.223257: E tensorflow/core/framework/dataset.cc:580] UNIMPLEMENTED: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2023-09-28 23:30:00.223323: E tensorflow/core/framework/dataset.cc:584] UNIMPLEMENTED: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.


### Calcular métricas:

In [42]:
metrics.log_loss(actuals, predictions)

0.02655661782230954

In [43]:
metrics.accuracy_score(actuals, predictions)

0.9992632095993264

In [44]:
metrics.average_precision_score(actuals, predictions)

0.5797465127493853

## Visualizar curvas de entrenamiento y validación (Tensorboard)

In [45]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir $DIR/logs

## Guardar modelo

In [48]:
model.save(f'{URI}/models/{TIMESTAMP}/model')

INFO:tensorflow:Assets written to: gs://statmike-mlops-349915/05/05/models/20230327115749/model/assets


INFO:tensorflow:Assets written to: gs://statmike-mlops-349915/05/05/models/20230327115749/model/assets


## Implementación de modelo (API)

### Registrar modelo en Vertex AI

Verificar que modelo no haya sido agregado anteriormente.   
Agregar modelo a Vertex AI.

In [60]:
modelmatch = aiplatform.Model.list(filter = f'display_name={SERIES}_{EXPERIMENT} AND labels.series={SERIES} AND labels.experiment={EXPERIMENT}')

upload_model = True
if modelmatch:
    print("Modelo ya existe")
    upload_model = False
else:
    print('Agregando modelo al registro')
    parent_model = ''

if upload_model:
    model = aiplatform.Model.upload(
        display_name = f'{SERIES}_{EXPERIMENT}',
        model_id = f'model_{SERIES}_{EXPERIMENT}',
        parent_model =  parent_model,
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/models/{TIMESTAMP}/model",
        is_default_version = True,
        version_aliases = [RUN_NAME],
        version_description = RUN_NAME,
        labels = {'series' : f'{SERIES}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}        
    )

Model Already in Registry:
Loading model as new default version.
Creating Model


INFO:google.cloud.aiplatform.models:Creating Model


Create Model backing LRO: projects/1026793852137/locations/us-central1/models/model_05_05/operations/1395049152047480832


INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/1026793852137/locations/us-central1/models/model_05_05/operations/1395049152047480832


Model created. Resource name: projects/1026793852137/locations/us-central1/models/model_05_05@13


INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/1026793852137/locations/us-central1/models/model_05_05@13


To use this Model in another session:


INFO:google.cloud.aiplatform.models:To use this Model in another session:


model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/model_05_05@13')


INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/model_05_05@13')


>**Note** on Version Aliases:
>Expectation is a name starting with `a-z` that can include `[a-zA-Z0-9-]`
>
>**Retrieve a Model Resource**
>[aiplatform.Model()](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model)
>```Python
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}') # retrieves default version
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}@time-{TIMESTAMP}') # retrieves specific version
model = aiplatform.Model(model_name = f'model_{SERIES}_{EXPERIMENT}', version = f'time-{TIMESTAMP}') # retrieves specific version
```

### Crear endpoint para hacer peticiones de predicciones

Verificar si *endpoint* ya existe.   
Crear *endpoint* (vacio por el momento).

In [77]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={SERIES}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{SERIES}",
        labels = {'series' : f"{SERIES}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")
    
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

Endpoint Exists: projects/1026793852137/locations/us-central1/endpoints/5876762107113897984
Review the Endpoint in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/endpoints/5876762107113897984?project=statmike-mlops-349915


In [78]:
endpoint.display_name

'05'

In [93]:
deployed_models = endpoint.list_models()
#deployed_models

### Desplegar modelo a endpoint
Agregar modelo en registro de Vertex AI al *endpoint* creado.

In [95]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = model.display_name,
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)


Deploying model with 100% of traffic...
Deploying Model projects/1026793852137/locations/us-central1/models/model_05_05 to Endpoint : projects/1026793852137/locations/us-central1/endpoints/5876762107113897984


INFO:google.cloud.aiplatform.models:Deploying Model projects/1026793852137/locations/us-central1/models/model_05_05 to Endpoint : projects/1026793852137/locations/us-central1/endpoints/5876762107113897984


Deploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/5876762107113897984/operations/4239072316731949056


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/5876762107113897984/operations/4239072316731949056


Endpoint model deployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/5876762107113897984


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/5876762107113897984


## Predicciones en línea

### Extraer datos para hacer predicciones

Se extraen 10 datos de *test*.

In [99]:
n = 10
pred = bq.query(
    query = f"""
        SELECT * EXCEPT({VAR_TARGET}, {VAR_OMIT}, splits)
        FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}
        WHERE splits='TEST'
        LIMIT {n}
        """
).to_dataframe()

In [100]:
pred

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,35337,1.092844,-0.013230,1.359829,2.731537,-0.707357,0.873837,-0.796130,0.437707,0.396770,...,-0.240428,0.037603,0.380026,-0.167647,0.027557,0.592115,0.219695,0.036970,0.010984,0.0
1,60481,1.238973,0.035226,0.063003,0.641406,-0.260893,-0.580097,0.049938,-0.034733,0.405932,...,-0.265080,-0.060003,-0.053585,-0.057718,0.104983,0.537987,0.589563,-0.046207,-0.006212,0.0
2,139587,1.870539,0.211079,0.224457,3.889486,-0.380177,0.249799,-0.577133,0.179189,-0.120462,...,-0.374356,0.196006,0.656552,0.180776,-0.060226,-0.228979,0.080827,0.009868,-0.036997,0.0
3,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,-0.923275,-0.545992,-0.252324,-1.171627,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0
4,165236,2.180149,0.218732,-2.637726,0.348776,1.063546,-1.249197,0.942021,-0.547652,-0.087823,...,-0.250653,0.234502,0.825237,-0.176957,0.563779,0.730183,0.707494,-0.131066,-0.090428,0.0
5,62606,1.199408,0.352007,0.379645,1.372017,0.291347,0.524919,-0.117555,0.132907,-0.935169,...,-0.042979,-0.050291,-0.126609,-0.022218,-0.599026,0.258188,0.928721,-0.058988,-0.008856,0.0
6,90719,1.937447,0.337882,-0.000630,3.816486,0.276515,1.079842,-0.730626,0.197353,1.137566,...,-0.315667,-0.038376,0.208914,0.160189,-0.015145,-0.162678,-0.000843,-0.018178,-0.039339,0.0
7,113350,1.891900,0.401086,-0.119983,4.047500,0.049952,0.192793,-0.108512,-0.040400,-0.390391,...,-0.267639,0.094177,0.613712,0.070986,0.079543,0.135219,0.128961,0.003667,-0.045079,0.0
8,156499,0.060003,1.461355,0.378915,2.835455,1.626526,-0.164732,1.551858,-0.412927,-1.735264,...,-0.175275,0.042293,0.277536,-0.123379,1.081552,-0.053079,-0.149809,-0.314438,-0.216539,0.0
9,73902,-1.859260,2.158799,1.085671,2.615483,0.246660,2.133925,-1.569015,-2.612353,-1.312509,...,0.590142,-0.867178,-0.700479,0.231972,-1.374527,0.140285,0.128806,0.153606,0.092042,0.0


In [101]:
newobs = pred.to_dict(orient = 'records')
#newobs[0]

### Obtener predicciones usando un cliente en Python

In [104]:
prediction = endpoint.predict(instances = newobs[0:1])
prediction

Prediction(predictions=[[0.993294418, 0.00670557469]], deployed_model_id='7857939123854639104', model_version_id='12', model_resource_name='projects/1026793852137/locations/us-central1/models/model_05_05', explanations=None)

In [105]:
prediction = endpoint.predict(instances = newobs)
prediction

Prediction(predictions=[[0.993294418, 0.00670557469], [0.998874247, 0.00112578226], [0.996102333, 0.0038977135], [0.999988675, 1.13643564e-05], [0.999644279, 0.000355680444], [0.993244469, 0.0067555015], [0.9964059, 0.00359401572], [0.992631078, 0.00736896461], [0.996538043, 0.00346192438], [0.999423862, 0.00057616405]], deployed_model_id='7857939123854639104', model_version_id='12', model_resource_name='projects/1026793852137/locations/us-central1/models/model_05_05', explanations=None)

In [106]:
prediction.predictions[0]

[0.993294418, 0.00670557469]

In [107]:
np.argmax(prediction.predictions[0])

0

### Obtener predicciones con petición REST

In [108]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": newobs[0:1]}))

In [109]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      0.993294418,
      0.00670557469
    ]
  ],
  "deployedModelId": "7857939123854639104",
  "model": "projects/1026793852137/locations/us-central1/models/model_05_05",
  "modelDisplayName": "05_05",
  "modelVersionId": "12"
}


### Obtener predicciones con gcloud (CLI)

In [110]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.993294418, 0.00670557469]]
